# Assignment 12

### 1.复习上课内容

### 2.回答以下理论问题

#### 1. 请写一下TF-IDF的计算公式

    TF: 某个词在该文段中出现的比例
    IDF: 所有文段数量与出现某个词的文段数量之间的比值
    TF-IDF: TF与IDF取log相乘

#### 2. LDA算法的基本假设是什么？

    每个文本的主题服从某个分布；每个主题下的词服从某个分布

#### 3. 在TextRank算法中构建图的权重是如何得到的？

    先取滑动窗口比如3，按顺序每连续3个词算相互连接的，其中每两个词之间的权重w由他们的词向量计算余弦相似度得到；
    开始将每个词在图中的权重WS统一初始化，然后进行迭代更新，对某个词的更新方法为找到与之相连接的所有词，分别计算每个w与连接词所有出度w之和的比例再乘上连接词各自的WS，一般还会取一个系数d，迭代到稳定为止

#### 4. 什么是命名实体识别？ 有什么应用场景？

    提取出文段中特定词所属的类别词性，如人名、机构名、地名等专有名词和有意义的时间等；问答系统，关系抽取，依存分析等

#### 5.NLP主要有哪几类任务 ？

    分类任务：文本分类，情感分析
    生成任务：机器翻译，问题回答，文章摘要

### 3.实践题

#### 3.1 手动实现TextRank算法 (在新闻数据中随机提取100条新闻训练词向量和做做法测试）

 提示：
 1. 确定窗口，建立图链接。   
 2. 通过词向量相似度确定图上边的权重
 3. 根据公式实现算法迭代(d=0.85)

In [43]:
from gensim.models import Word2Vec
import pandas as pd
import jieba
import jieba.posseg as jp
import random
import re
import numpy as np
from operator import itemgetter
from itertools import combinations
import warnings
from collections import defaultdict
from scipy.spatial.distance import cosine

warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('sqlResult_1558435.csv', encoding='gb18030')

In [3]:
news = data['content'][np.random.randint(len(data), size=100)]
len(news)

100

In [4]:
stopwords = [w.strip() for w in open('哈工大停用词表.txt', encoding='utf-8') if w.strip() != '']

In [48]:
def token(string):
    return re.findall(r'[\d|\w]+',string)

In [64]:
%%time

flags = ('n', 'nr', 'ns', 'nt', 'eng', 'v', 'd')
words_ls = []
for text in news.astype(str):
    words = [w.word for w in jp.cut(''.join(token(text))) 
             if w.flag in flags and w.word not in stopwords and w.word != 'n']
    words_ls.append(words)

CPU times: user 1.24 s, sys: 4.49 ms, total: 1.25 s
Wall time: 1.25 s


In [66]:
word2vec_model = Word2Vec(words_ls, min_count=1, iter=10)

In [38]:
def get_important_words_by_textrank(cutwords:list, word2vec_model, n=5):
    '''cutwords的输入为words_ls中的一个文段分词结果，取前n个重要的词，滑动窗口默认取3'''

    n_words = len(cutwords)
    w2w_link = defaultdict(set)
    w2w_weight = {}
    for i in range(n_words-3+1):
        word_a, word_b, word_c = cutwords[i:i+3]
        w2w_link[word_a].update([word_b, word_c])
        w2w_link[word_b].update([word_a, word_c])
        w2w_link[word_c].update([word_a, word_b])
        w2w_weight[(word_a, word_b)] = cosine(word2vec_model[word_a], word2vec_model[word_b])
        w2w_weight[(word_a, word_c)] = cosine(word2vec_model[word_a], word2vec_model[word_c])
        w2w_weight[(word_b, word_c)] = cosine(word2vec_model[word_b], word2vec_model[word_c])
        
    words = set(cutwords)
    WS = {}
    for w in words:
        WS[w] = 1/n_words
    
    flag = True
    while flag:
        new_WS = {}
        for wi in words:
            ws_wi = 0
            for wj in w2w_link[wi]:
                denominator = sum([w2w_weight[(wj, x)] if (wj, x) in w2w_weight 
                                   else w2w_weight[(x, wj)] for x in w2w_link[wj]])
                if (wi, wj) in w2w_weight:
                    ws_wi += w2w_weight[(wi, wj)] / denominator * WS[wj]
                else:
                    ws_wi += w2w_weight[(wj, wi)] / denominator * WS[wj]
                    
            new_WS[wi] = 0.15 + 0.85*ws_wi
            
        if all([np.abs(new_WS[wi] - WS[wi]) < 0.001 for wi in words]):
            flag = False
            
        for wi in words:
            WS[wi] = new_WS[wi]
    
    rank = sorted(WS.items(), key=itemgetter(1), reverse=True)[:n]
    important_words = []
    for (w, _) in rank:
        important_words.append(w)
    
    return important_words

In [67]:
get_important_words_by_textrank(words_ls[0], word2vec_model, n=5)

['问题', '山西省', '治污', '整改', '山西']

In [68]:
get_important_words_by_textrank(words_ls[1], word2vec_model, n=5)

['舞会', '奥斯卡', '艺术', '慈善', '时尚界']

In [69]:
get_important_words_by_textrank(words_ls[2], word2vec_model, n=5)

['甘冈', '南锡', '迪亚', '甲级联赛', '足球']

### 3.2 使用词向量和k-means的方法寻找关键词

提示：   
1.使用3.1训练好的词向量   
2.可使用sklearn等机器学习库

In [70]:
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error

#### 得到每个分类中心后，将该分类下的词与该中心计算距离并排序，取最近的词为该中心的代表词，请问这里计算距离用cosine与mean_squared_error哪个更合适呢？

In [84]:
def get_important_words_by_kmeans(cutwords:list, word2vec_model, n=5):
    words_vec = []
    for w in cutwords:
        words_vec.append(word2vec_model[w])
        
    kmeans_model = KMeans(n_clusters=n)
    kmeans_model.fit(np.array(words_vec))
    label_of_words = defaultdict(list)
    
    for w, label in zip(cutwords, kmeans_model.labels_):
        label_of_words[label].append(w)
    
    important_words = []
    for label, words in label_of_words.items():
        center = kmeans_model.cluster_centers_[label]
        distance2center = {}
        for word in words:
            distance2center[word] = mean_squared_error(word2vec_model[word], center)
        rank = sorted(distance2center.items(), key=itemgetter(1), reverse=False)[0]
        important_words.append(rank[0])
        
    return important_words

In [96]:
get_important_words_by_kmeans(words_ls[0], word2vec_model, n=5)

['问题', '太原市', '治理', '进行', '整改']

In [97]:
get_important_words_by_kmeans(words_ls[1], word2vec_model, n=5)

['新华社', '举行', '外代', '博物馆', '舞会']

In [98]:
get_important_words_by_kmeans(words_ls[2], word2vec_model, n=5)

['新华社', '队', '外代', '南锡', '胜']

### 3.3  提取新闻人物里的对话。(使用以上提取小数据即可）

提示：    
1.寻找预料里具有表示说的意思。    
2.使用语法分析提取句子结构。    
3.检测谓语是否有表示说的意思。

In [88]:
import pyltp
from gensim.models import Word2Vec
import jieba
from gensim.models.word2vec import LineSentence
from pyltp import SentenceSplitter, NamedEntityRecognizer, Postagger, Parser, Segmentor
from gensim import models
import numpy as np

In [115]:
text = jieba.lcut(news.tolist()[68])
text = ''.join([w for w in text if w not in ['\\', 'n']])

In [116]:
text

'n2017年5月10日，在韩国首尔青瓦台总统府，新当选总统文在寅在记者会上发言。（新华社记者姚琪琳摄）新华社北京5月24日电（记者杨舒怡）韩国总统文在寅22日下令对一个庞大的水利项目启动调查，怀疑该项目决策过程不透明、招投标环节可能存在“猫腻”。这个大型水利项目牵涉4条河流的数十座水坝和近百个水库，总耗资22万亿韩元（约合1349.4亿元人民币），备受韩国民众关注。【涉嫌违规】韩国《中央日报》23日援引韩国负责社会事务的官员金秀炫（音译）的话报道，韩国总统文在寅22日下令对牵涉汉江、锦江、洛东江、荣山江4条河流的一个大型水利项目展开审计工作，以查明其决策过程和实施环节是否存在“猫腻”。该项目启动于2009年，2011年竣工，总共修建了21座水坝和96个水库，以期减少洪灾和旱灾影响，以及净化河流。不过，该项目招致不少质疑，被指未能达到预期目标。据金秀炫介绍，不少人怀疑该项目仓促上马，在政府决策和后续实施过程中“不符常规”。文在寅要求，此次审计结果将通过白皮书形式发表，希望能够促进今后决策透明、执行有力。按照金秀炫的说法，“审计中若查出任何违规和腐败情形，都将依法处置”。【多次挨查】这一水利项目引发争议，过去几年中已经受过3次调查。前两次调查均在李明博执政时期，分别发生在2011年1月、2013年1月。第一次调查并未查出明显违规；第二次调查则指出，该项目在水质管理方面存在一些问题。第三次调查是在2013年7月，即朴槿惠执政时期。这次调查发现不少问题，例如招投标过程存在共谋、报价虚高等。韩国总统府青瓦台一名官员22日总结道：“李明博总统时期的两次审计对该项目的决策本身并未提出质疑；而朴槿惠总统时期的那次审计主要关注多家建筑公司的共谋活动。”金秀炫22日透露，李明博执政期间的部分官员卷入这一水利项目，今后可能会受到刑事调查。【承诺改革】由于在水质管理等方面存在缺陷，这一水利项目今后的命运在韩国社会引发讨论。韩国政府打算成立一个专家组，对该项目展开综合评估。其中，部分水坝的去留问题将在2018年年底予以决定。金秀炫说，韩国政府还考虑将环境部负责水质管理的部门与国土交通部合并，希望以此提升办事效率。文在寅今年竞选总统期间曾表示，上台后将努力革除历届政府积弊，包括查处国防采购、兴修水利等领域的贪腐案。韩国《中央日报》评论，文在寅22日宣布调查水利项目的潜在问题，表明其着手兑现竞

In [117]:
present_word = ['说', '介绍', '总结', '总结道', '透露']

In [128]:
cws_model = "ltp_data_v3.4.0/cws.model"
pos_model = "ltp_data_v3.4.0/pos.model"
par_model = "ltp_data_v3.4.0/parser.model"
ner_model = "ltp_data_v3.4.0/ner.model"

In [119]:
def get_word_list(sentence,model):
    #得到分词
    segmentor = Segmentor()
    segmentor.load(model)
    word_list = list(segmentor.segment(sentence))
    segmentor.release()
    return word_list

def get_postag_list(word_list,model):
    #得到词性标注
    postag = Postagger()
    postag.load(model)
    postag_list = list(postag.postag(word_list))
    postag.release()
    return postag_list

def get_parser_list(word_list,postag_list,model):
    #得到依存关系
    parser = Parser()
    parser.load(model)
    arcs = parser.parse(word_list,postag_list)
    arc_list = [(arc.head,arc.relation) for arc in arcs]
    parser.release()
    return arc_list

In [129]:
word_list = get_word_list(text, cws_model)

In [130]:
postag_list = get_postag_list(word_list, pos_model)

In [131]:
parser_list = get_parser_list(word_list, postag_list, par_model)

In [132]:
for i in range(len(word_list)):
    print(word_list[i], parser_list[i])

n2017 (2, 'ATT')
年 (4, 'ATT')
5月 (4, 'ATT')
10日 (13, 'ADV')
， (4, 'WP')
在 (13, 'ADV')
韩国 (8, 'ATT')
首尔 (9, 'ATT')
青瓦台 (10, 'ATT')
总统府 (6, 'POB')
， (6, 'WP')
新 (13, 'ADV')
当选 (81, 'ATT')
总统 (15, 'ATT')
文 (13, 'VOB')
在 (21, 'ADV')
寅 (21, 'SBV')
在 (21, 'ADV')
记者会 (20, 'ATT')
上 (18, 'POB')
发言 (13, 'COO')
。 (13, 'WP')
（ (27, 'WP')
新华社 (25, 'ATT')
记者 (26, 'ATT')
姚琪琳 (27, 'SBV')
摄 (13, 'COO')
） (27, 'WP')
新华社 (33, 'SBV')
北京 (33, 'ADV')
5月 (32, 'ATT')
24日 (33, 'ADV')
电 (13, 'COO')
（ (36, 'WP')
记者 (36, 'ATT')
杨舒怡 (33, 'CMP')
） (36, 'WP')
韩国 (39, 'ATT')
总统 (40, 'ATT')
文 (44, 'SBV')
在 (44, 'ADV')
寅 (43, 'ATT')
22日 (41, 'POB')
下令 (33, 'VOB')
对 (51, 'ADV')
一个 (50, 'ATT')
庞大 (50, 'ATT')
的 (47, 'RAD')
水利 (50, 'ATT')
项目 (45, 'POB')
启动 (44, 'VOB')
调查 (51, 'VOB')
， (44, 'WP')
怀疑 (44, 'COO')
该 (56, 'ATT')
项目 (58, 'ATT')
决策 (58, 'ATT')
过程 (63, 'ATT')
不 (60, 'ADV')
透明 (63, 'ATT')
、 (62, 'WP')
招投标 (63, 'ATT')
环节 (65, 'SBV')
可能 (65, 'ADV')
存在 (54, 'VOB')
“ (67, 'WP')
猫腻 (65, 'VOB')
” (67, 'WP')
。 (13, 'WP')


In [135]:
for i in range(len(word_list)):
    if word_list[i] in present_word:
        print(word_list[i], parser_list[i])

介绍 (219, 'POB')
总结 (100, 'COO')
透露 (411, 'COO')
说 (411, 'COO')
